## Statistics 504 Homework #1 <br>
Fall 2015 <br>
Erich Seamon <br>
University of Idaho <br>
erichs@uidaho.edu <br>

## Exercise 1
### Write a program that reads a file, breaks each line into words, strips whitespace and punctuation from the words, and converts them to lowercase.

#### Solution:  

Below is a set of defined functions that accomplish Exercise 1.

#### Functions used as part of Exercise 1: 

-process_file - a function that calls the filename that will be set in __main__ - defines a binary skip_header argument, and then processes each line in the datafile. <br>
-skip_gutenberg_header - a function that skips the header of a file <br>
-print_line - goes thru each line and breaks each word apart, strips out puncuation<br>

These three functions complete Exercise 1.

In [1280]:
import operator
import string
import random
import urllib2
from string import digits
import re


##### Loading the king james file from a url

In [1281]:
book = urllib2.urlopen('https://www.reacchpna.org/files/7614/4236/8760/kingjames.txt').read()


##### splitting up the king james bible into Old and New Testaments, taking the header and footer off

In [1282]:
f = book.split('\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n')[1]
lines = f.rpartition('The New Testament of the King James Bible')
NewT = f.split('***')[1]


OldTestament = lines[0]
g = lines[2]
NT = NewT.rpartition('End of the Project Gutenberg EBook of The King James Bible') 
NewTestament = NT[0]

# sends output to a variable

h = open('NewTestament.txt','w')
print >>h, NewTestament

i = open('OldTestament.txt','w')
print >>i, OldTestament

##### Function definitions

In [1283]:
def process_file(filename, skip_header):
    """Makes a histogram that contains the words from a file.

    filename: string
    skip_header: boolean, whether to skip the Gutenberg header
   
    Returns: map from each word to the number of times it appears.
    """
    hist = {}
    fp = file(filename)

    if skip_header:
        skip_gutenberg_header(fp)

    for line in fp:
        process_line(line, hist)
    return hist

In [1284]:
def skip_gutenberg_header(fp):
    """Reads from fp until it finds the line that ends the header.

    fp: open file object
    """
    for line in fp:
        if line.startswith('*END*THE SMALL PRINT!'):
            break

In [1285]:
def process_line(line, hist):
    """Adds the words in the line to the histogram.

    Modifies hist.

    line: string
    hist: histogram (map from word to frequency)
    """
    # replace hyphens with spaces before splitting
    line = line.replace('-', ' ')
    
    #tried the below to replace the colon in advance, then replace the full numerical string.  
    line = line.replace(':', '')

    
    for word in line.split():
        # remove punctuation and convert to lowercase

        word = word.strip(string.whitespace)
        word = word.strip()
        word = word.strip(string.punctuation)
        #word = word.strip(string.punctuation)        
        word = word.lower()
        word = word.replace("'", "") #added to remove extraneous leftover apostrophes
        
        #1. tried this - induces white space
        word = re.sub("\S*\d\S*", "ERICH", word).strip()
        
        word = word.strip("ERICH" + string.whitespace)
        
        #2. tried this - again induces a white space - seemingly for the colon.  One white space for every chapter and verse.
        #word = word.strip(string.digits)
        
        #3. again, induces white space, same number as the above. 
        #word = word.strip()


        
        
        # update the histogram
        hist[word] = hist.get(word, 0) + 1

In [1286]:
def most_common(hist, trim):
    """Makes a list of the key-value pairs from a histogram and
    sorts them in descending order by frequency.
    
    threshold = the cutoff threshold for frequency occurances"""
    
    t = []
    for key, value in hist.items(): 
        if value >= trim:  #--added by es to enable k-fold list
            t.append((value, key))
    t.sort()
    t.reverse() 
    
    return t



## Exercise 2
### Go to Project Gutenberg (http://gutenberg.org) and download The King James Bible available at http://www.gutenberg.org/cache/epub/10/pg10.txt in plain text format. 

Modify your program from the previous exercise to read The King James Bible you downloaded, skip over the header information at the beginning of the file, and process the rest of the words as before. 

Then modify the program to count the total number of words in the Old Testament, and print out the number of times each word is used.  alongside the word if the word is used more than k times (where k as a user input parameter) in decreasing order. Print out the output when k=100.  (list of all the words – with their count beside them – with a cutoff of 100 (k).  K is input).  

Repeat the above for the New Testament. 


#### Solution:  

Building upon the solution provided in Exercise 1 - below is:

-a set of defined functions that accomplish Exercise 2, as well as: <br>
-a __main__ code section that calls all functions and sets the data source. <br>

The output below counts the total number of words in the Old Testament, with a frequency of 100 cutoff.

#### Functions used as part of Exercise 2: <br>

-most_common - determines the most frequent words.  This function is where the k threshold was set. <br>
-print_most_common - prints the most common words determined from most_common <br>
-subract - Removes any words that we might not want to include - such as prepositions, pronouns, etc. <br>
-total_words - Does a count of total words <br>

In [1287]:
def print_most_common(hist, num=10):
    """Prints words that occur at least K times in a histgram and their frequencies.
    in this instance, 100 is used.
    
    hist: histogram (map from word to frequency
    threshold: the lowest frequency of words to show.
    """
    
    t = most_common(hist)
    print 'The most common words are:'
    for freq, word in t[:num]:
        print word, '\t', freq

In [1288]:
def subtract(d1, d2):
    """Returns a dictionary with all keys that appear in d1 but not d2.

    d1, d2: dictionaries
    """
    res = {}
    for key in d1:
        if key not in d2:
            res[key] = None
    return res

In [1289]:
def total_words(hist):
    """Returns the total of the frequencies in a histogram."""
    return sum(hist.values())

In [1290]:
def different_words(hist):
    """Returns the number of different words in a histogram."""
    return len(hist)

In [1291]:
def random_word(hist):
    """Chooses a random word from a histogram.

    The probability of each word is proportional to its frequency.
    """
    t = []
    for word, freq in hist.items():
        t.extend([word] * freq)

    return random.choice(t)

##### Run all the functions for the Old Testament

In [1292]:
if __name__ == '__main__':
    hist = process_file('OldTestament.txt', skip_header=False)
    print 'Total number of words:', total_words(hist)
    print 'Number of different words:', different_words(hist)
    hist.pop("", None) #-added to remove white space oddness

    t = most_common(hist, 100)
    print 'The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:'
    for freq, word in t:
        print word, '\t', freq
        #print word, freq

    #print "\n\nHere are some random words from the book"
    #for i in range(100):
        #print random_word(hist),


Total number of words: 632595
Number of different words: 10755
The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:
the 	52992
and 	40975
of 	28548
to 	10207
in 	9767
that 	9152
shall 	8423
he 	7347
lord 	7119
his 	7036
for 	6689
i 	6669
unto 	6551
a 	6268
they 	5411
be 	5296
it 	4852
is 	4820
them 	4730
with 	4643
thou 	4617
not 	4533
all 	4490
him 	4193
thy 	4184
my 	3749
their 	3445
thee 	3344
will 	3281
me 	3253
was 	3172
god 	3089
said 	2938
from 	2910
which 	2860
have 	2820
israel 	2490
upon 	2485
as 	2480
ye 	2424
but 	2323
out 	2267
king 	2171
were 	1989
are 	1987
up 	1971
this 	1968
son 	1948
people 	1908
when 	1862
house 	1818
by 	1815
man 	1744
there 	1732
hath 	1707
her 	1690
land 	1668
children 	1645
came 	1587
then 	1581
shalt 	1509
before 	1503
against 	1457
on 	1437
day 	1418
an 	1385
come 	1384
had 	1383
you 	1373
into 	1371
one 	1360
your 	1323
so 	1314
hand 	1308
men 	1267
go 	1240
at 	1180
also 	1170
let 	1169
made 	1119
even 	1118


### Exercise 3 - Below we run the subtract function to eliminate any words in our words list from the OLD TESTAMENT.  This list happens to be a list of prepositions, pronouns, conjunctions, and articles for removal.


In [1293]:
    words = process_file('/git/stat504-python/STAT504-HW1/HW1-wordlist.txt', skip_header=True)

    diff = subtract(hist, words)
    print "The words in the OLD TESTAMENT that are not prepositions, pronouns, conjunctions, or articles:"
    for word in diff.keys():
        print word,

The words in the OLD TESTAMENT that are not prepositions, pronouns, conjunctions, or articles:
giddel yellow four aijalon spiders hanging deserveth sevens looking shammuah rouse lord taanathshiloh meadows treasuries slothful hough intercessor cozbi jeroboams figs bringing eshek chesed uttering scholar elihu lionlike brutish woods rebekahs ahinoam provided inheritances hanan lovest ligure specially nigh tired coniah suppliants pulse millet tires second valiant holon scraped haughtily errors thunder fingers bethphelet shocks crouch gentiles herb pibeseth error here laadah reported hiram gileads hers substance mahalah cainan climbed sighest controversy dimness cheereth kindly golden whoso divide birthright athaliah lengthen brought darkon remnant wipeth forborn declareth woundedst siddim clifts reverenced therefore violently strike turtledoves until fatling holy hereby chenaniah hurt glass huri rescueth midst hold adnah addon obededom locked melchishua pursue blade jekuthiel maoch teman c

### Repeating Exercise 1,2, and 3 for the New Testament

#### Exercise 2 - Run all the functions for the NEW TESTAMENT

In [1294]:
if __name__ == '__main__':
    hist = process_file('NewTestament.txt', skip_header=False)
    print 'Total number of words:', total_words(hist)
    print 'Number of different words:', different_words(hist)
    hist.pop("", None) #-added to remove white space oddness
    
    t = most_common(hist, 100)
    
    print 'The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:'
    for freq, word in t:
        print word, '\t', freq
        #print word, freq

    #print "\n\nHere are some random words from the book"
    #for i in range(100):
        #print random_word(hist),

Total number of words: 188560
Number of different words: 6003
The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:
the 	11031
and 	10721
of 	6122
that 	3760
to 	3373
he 	3072
in 	2900
him 	2466
unto 	2446
for 	2281
i 	2185
is 	2169
not 	2063
they 	1965
a 	1909
be 	1717
them 	1700
but 	1669
ye 	1559
which 	1553
his 	1437
shall 	1415
with 	1369
god 	1356
was 	1350
it 	1277
you 	1244
all 	1130
have 	1084
said 	1061
as 	1040
jesus 	983
when 	972
are 	963
we 	869
man 	869
thou 	857
me 	843
this 	817
by 	810
were 	783
from 	732
things 	729
lord 	711
into 	644
had 	643
my 	619
one 	607
if 	602
also 	599
then 	588
come 	587
on 	574
there 	567
hath 	557
will 	555
christ 	555
out 	508
came 	506
us 	489
their 	487
thee 	483
no 	470
your 	455
saying 	455
say 	428
son 	422
thy 	416
do 	416
up 	409
what 	408
now 	407
these 	392
at 	390
men 	386
who 	378
so 	375
should 	366
our 	366
therefore 	356
father 	352
let 	342
went 	338
or 	324
day 	316
an 	314
saith 	307
who

#### Exercise 3 - Below we run the subtract function to eliminate any words in our words list from the NEW TESTAMENT.  This list happens to be a list of prepositions, pronouns, conjunctions, and articles for removal.


In [1295]:
   words = process_file('/git/stat504-python/STAT504-HW1/HW1-wordlist.txt', skip_header=False)
   diff = subtract(hist, words)
   print "The words in the NEW TESTAMENT that are not prepositions, pronouns, conjunctions, or articles:"
   for word in diff.keys():
        print word,

The words in the NEW TESTAMENT that are not prepositions, pronouns, conjunctions, or articles:
writings foul four railing cyprus looking lord slothful rashly figs bringing prize broiled specially nigh complainers second valiant consenting sailed errors presbytery thunder schism paphos lowliness increasing gentiles eunice unseemly herd reported substance cainan climbed stephanas controversy thamar golden divide sopater brought remnant stern ragau phalec overshadow strike turtledoves holy hereby hurt glass midst hold blade leaped example vanities unjust household kingdoms want reprove groaned travel damage hot acknowledgeth salmone saron festus wrong worshippeth extortion calves scourge endeavoured maids wind wine rewarder fidelity wrought fit heretofore abominations savourest jambres fig urias hidden easier glorify grievous thrones raiseth sixteen silver rumour vipers addicted burial woes concord garment whit blinded borne sosipater urbane foundation assured written threatened grapes cr